In [ ]:
#!pip install geopy

In [ ]:
from geopy.geocoders import Nominatim
import pandas as pd
import time

# Lee el archivo IPC2002-2024.csv separado por tabulador
df_original = pd.read_csv('IPC2002-2024.csv', sep='\t')

# Convertimos la columna 'Total' a tipo float (reemplazando comas por puntos)
df_original['Total'] = df_original['Total'].str.replace(',', '.')
df_original['Total'] = df_original['Total'].astype(float)

# Hacemos una copia profunda del DataFrame
df = df_original.copy(deep=True)

# Limpiamos los primeros dígitos de la columna 'Provincias'
df['provincias_cleaned'] = df['Provincias'].str.replace(r'^\d+\s*', '', regex=True)


# Eliminamos filas donde la provincia sea 'Nacional'
df = df[df['provincias_cleaned'] != 'Nacional']

# Buscamos las coordenadas de cada provincia

In [5]:
geolocator = Nominatim(user_agent="geoapi")
geo_cache = {}  # Diccionario para almacenar los resultados de geocodificación
latitudes = []
longitudes = []

for prov in df['provincias_cleaned']:
    if prov not in geo_cache:  # Si la provincia no está en el diccionario, buscarla
        location = geolocator.geocode(prov, country_codes="ES")
        if location:
            geo_cache[prov] = (location.latitude, location.longitude)
        else:
            geo_cache[prov] = (None, None)
        time.sleep(1)  # Para no saturar el servicio
    latitudes.append(geo_cache[prov][0])
    longitudes.append(geo_cache[prov][1])

df['lat'] = latitudes
df['lon'] = longitudes


#concatena a el dataframe original la latitud y longitud
df_original = pd.concat([df_original, df[['lat', 'lon']]], axis=1)
df_original.to_json("IPC2002-2024.json", orient='records', force_ascii=False)